# Solving Hyperbolic PDEs via Finite Differences

We want to solve
$$\partial_t^2\phi = \partial_x^2\phi+\partial_y^2\phi$$
$$\phi|_{\partial\Omega}=0,$$
where $\phi = \phi(t, x, y)$ with $(t, x, y)\in\Omega$.

How to solve this easily?
Define $\psi:=\partial_t\phi$. Then
$$\partial_t\phi = \psi$$
$$\partial_t\psi = \partial_x^2\phi+\partial_y^2\phi.$$

In [3]:
using CairoMakie
import Pkg; Pkg.add("DifferentialEquations")
using DifferentialEquations
using LinearAlgebra

    Updating registry at `C:\Users\hpark1\.julia\registries\General.toml`
   Resolving package versions...
   Installed Sundials_jll ───────────────────── v5.2.2+0
   Installed TimerOutputs ───────────────────── v0.5.23
   Installed MaybeInplace ───────────────────── v0.1.1
   Installed StaticArrays ───────────────────── v1.9.2
   Installed Polyester ──────────────────────── v0.7.9
   Installed RecursiveArrayTools ────────────── v3.8.0
   Installed DifferentialEquations ──────────── v7.12.0
   Installed NonlinearSolve ─────────────────── v3.5.3
   Installed FunctionWrappers ───────────────── v1.1.3
   Installed TriangularSolve ────────────────── v0.1.20
   Installed RandomNumbers ──────────────────── v1.5.3
   Installed SLEEFPirates ───────────────────── v0.6.42
   Installed Distances ──────────────────────── v0.10.11
   Installed Static ─────────────────────────── v0.8.9
   Installed SteadyStateDiffEq ──────────────── v2.0.1
   Installed BoundaryValueDiffEq ────────────── v5.6.0
   In

In [24]:
# Set up by defining the discrete grid for Omega space
npoints = 51 # we first started with 5
h = 2 / (npoints - 1) # grid spacing (division is by 2 because ...)

0.04

In [27]:
# It's convenient to put the whole state vector in a single vector!
U = zeros(npoints, npoints, 2) # will hold all field configurations at each point! (third argument is the third axis)

# More specifically, we define
function DifferentialEquations.init()
    # initial conditions
    x0 = 0.0
    y0 = -0.5
    W = 0.01
    
    U = zeros(npoints, npoints, 2)
    for j in 1:npoints, i in 1:npoints
        if i==1 || i==npoints || j==1 || j==npoints
            U[i,j,1]=0
            U[i,j,2]=0
        else
            x = -1 + (i-1)*h # at i=1, start from x = -1
            y = -1 + (j-1)*h # at j=1, start from y = -1
            phi = exp(-((x-x0)^2+(y-y0)^2)/2W)
            psi = 0
            U[i,j,1] = phi
            U[i,j,2] = psi
        end
    end
    return U
end

For finite differences, recall
$$f_i''=\frac{f_{i-1}-2f_i+f_{i+1}}{h^2}.$$

In [29]:
# right hand side function
function rhs(U, p, t)
    # need p, t parameters to plug into ODE package's function!
    Udot = similar(U)

    for j in 1:npoints, i in 1:npoints
        if i==1 || i==npoints || j==1 || j==npoints
            Udot[i,j,1]=0
            Udot[i,j,2]=0
        else
            phixx = (U[i-1,j,1]-2*U[i,j,1]+U[i+1,j,1])/h^2 # recall U's third axis had phi stored in index 1!!
            phiyy = (U[i,j-1,1]-2*U[i,j,1]+U[i,j+1,1])/h^2
            Udot[i,j,1] = U[i,j,2]
            Udot[i,j,2] = phixx + phiyy
        end
    end
    return Udot
end

# For finite differeces, it's more customary to evaluate matrix at the point, rather than storing the value!

rhs (generic function with 1 method)

In [30]:
U0 = init()
tspan = (0.0, 4.0) # time span
prob = ODEProblem(rhs, U0, tspan)
sol = solve(prob);
length(sol.t)

110

In [31]:
# Now let's plot
fig = Figure(size=(600, 600))
heatmap!([-1:h:+1, -1:h:-1], sol(0.0)[:,:,1];colorrange=(-1,1)) # init:stepsize:final
heatmap!([-1:h:+1, -1:h:-1], sol(3.0)[:,:,1];colorrange=(-1,1)) # init:stepsize:final
fig

# need to make sure aspect ratio is square! (which is NOT quite the figure size)

LoadError: `Makie.convert_arguments` for the plot type Heatmap and its conversion trait CellGrid() was unsuccessful.

The signature that could not be converted was:
::Vector{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, ::Matrix{Float32}

Makie needs to convert all plot input arguments to types that can be consumed by the backends (typically Arrays with Float32 elements).
You can define a method for `Makie.convert_arguments` (a type recipe) for these types or their supertypes to make this set of arguments convertible (See http://docs.makie.org/stable/documentation/recipes/index.html).

Alternatively, you can define `Makie.convert_single_argument` for single arguments which have types that are unknown to Makie but which can be converted to known types and fed back to the conversion pipeline.


^^^ Indeed, the function on the boundary is zero, at time 0, consistent with our chosen intial condition!

In [32]:
# Now let's plot
fig = Figure(size=(600, 600))
contourf!([-1:h:+1, -1:h:-1], sol(0.0)[:,:,1];colorrange=(-1,1)) # init:stepsize:final
contourf!([-1:h:+1, -1:h:-1], sol(3.0)[:,:,1];colorrange=(-1,1)) # init:stepsize:final
fig

# need to make sure aspect ratio is square!

LoadError: `Makie.convert_arguments` for the plot type Plot{Makie.contourf} and its conversion trait VertexGrid() was unsuccessful.

The signature that could not be converted was:
::Vector{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, ::Matrix{Float32}

Makie needs to convert all plot input arguments to types that can be consumed by the backends (typically Arrays with Float32 elements).
You can define a method for `Makie.convert_arguments` (a type recipe) for these types or their supertypes to make this set of arguments convertible (See http://docs.makie.org/stable/documentation/recipes/index.html).

Alternatively, you can define `Makie.convert_single_argument` for single arguments which have types that are unknown to Makie but which can be converted to known types and fed back to the conversion pipeline.


In [33]:
# Now let's plot
fig = Figure(size=(640, 400))
axis = Axis(fig[1,1])
lines!(axis, -1:h:+1, sol(0.0)[:,(npoints+1)+2,1]) # init:stepsize:final
lines!(axis, -1:h:+1, sol(0.5)[:,(npoints+1)+2,1]) # init:stepsize:final
lines!(axis, -1:h:+1, sol(1.0)[:,(npoints+1)+2,1]) # init:stepsize:final
lines!(axis, -1:h:+1, sol(1.5)[:,(npoints+1)+2,1]) # init:stepsize:final
lines!(axis, -1:h:+1, sol(2.0)[:,(npoints+1)+2,1]) # init:stepsize:final
fig

LoadError: BoundsError: attempt to access 51×51×2 Array{Float64, 3} at index [1:51, 54, 1]

## Effect of Resolution
twice the resolution, 8 times the operation time!

In [ ]:
# It's convenient to put the whole state vector in a single vector!
U = zeros(npoints, npoints, 2) # will hold all field configurations at each point! (third argument is the third axis)

# More specifically, we define
function init(npoints)
    h = 2/(npoints-1)
    # initial conditions
    x0 = 0.0
    y0 = -0.5
    W = 0.01
    
    U = zeros(npoints, npoints, 2)
    for j in 1:npoints, i in 1:npoints
        if i==1 || i==npoints || j==1 || j=npoints
            U[i,j,1]=0
            U[i,j,2]=0
        else
            x = -1 + (i-1)*h # at i=1, start from x = -1
            y = -1 + (j-1)*h # at j=1, start from y = -1
            phi = exp(-((x-x0)^2+(y-y0)^2)/2W)
            psi = 0
            U[i,j,1] = phi
            U[i,j,2] = psi
        end
    end
    return U
end

In [ ]:
# right hand side function
function rhs(U, p, t)
    npoints = length(U)
    # need p, t parameters to plug into ODE package's function!
    Udot = similar(U)

    for j in 1:npoints, i in 1:npoints
        if i==1 || i==npoints || j==1 || j=npoints
            Udot[i,j,1]=0
            Udot[i,j,2]=0
        else
            phixx = (U[i-1,j,1]-2*U[i,j,1]+U[i+1,j,1])/h^2 # recall U's third axis had phi stored in index 1!!
            phiyy = (U[i,j-1,1]-2*U[i,j,1]+U[i,j+1,1])/h^2
            Udot[i,j,1] = U[i,j,2]
            Udot[i,j,2] = phixx + phiyy
        end
    end
    return Udot
end

# For finite differeces, it's more customary to evaluate matrix at the point, rather than storing the value!

...julia: `end` command in square bracket looks at the last index point!

## Convergence

$$\left(\int_\Omega|f_n(t,x)-f^*(t,x)|^2 dx\right)^2 = O(h^2)$$

In logE vs logh plot, we'll see a plot with trend: ---\ \ \ \ \ \ \------10^-15 (floating point error---computer's minimum error)!

Finite differencing method has larger error than 10^-15